In [19]:
import ipyvuetify as v
from ipywidgets import widgets, Layout, HTML
from IPython.display import display, IFrame

from voilalibrary.vuetify import settings
settings.dark_mode      = False
settings.color_first    = '#53b6d3'
settings.color_second   = '#146a9b'
settings.button_rounded = True

from voilalibrary.vuetify import app, datePicker, button, treeview, switch, label, progress, selectSingle

from dateutil.relativedelta import relativedelta
from datetime import datetime
import calendar
from time import sleep

import requests
import json

api_key = 'eobzKN3RhAY0Mg7HFeNzARmbJURbtH3fd9h0D1OL'

In [15]:
# Formats one of the results of the APOD search by returning a v.Card containing small image + title + explanation with links
def formatResult(res, elevation=3, clickable=True):
    
    date = None
    
    def onclicked(widget, event, data):
        if not date is None:
            dp.date = date.strftime("%Y-%m-%d")
            g_app.tabs.v_model = 0
            on_click_tab('DAY-BY-DAY')
    
    if 'apod.nasa.gov/apod/ap' in res:
        pos = res.index('apod.nasa.gov/apod/ap') + len('apod.nasa.gov/apod/ap')
        datestr = res[pos:pos+6]
        date = datetime.strptime(datestr,'%y%m%d')
        
        image = ''
        if '/apod/calendar/' in res:
            pos1 = res.index('/apod/calendar/') + len('/apod/calendar/')
            pos2 = res[pos1:].index('>')
            image = 'https://apod.nasa.gov/apod/calendar/' + res[pos1:pos1+pos2]
        
        pos = res.index(' APOD:')
        res = '<img src=%s></img>' % image + res[pos:]
        
        card = v.Card(elevation=elevation, class_='pa-2 pb-1 ma-3 mb-1',
                      #children=[widgets.HTML('<p style="line-height: 21px; font-size: 14px;">' +
                      children=[widgets.HTML('<p style="line-height: 2.0vh; font-size: 1.5vh;">' +
                                             res.replace('src=/apod/calendar/','src=https://apod.nasa.gov/apod/calendar/')\
                                                .replace('<b> Explanation: </b>','')\
                                                .replace(' APOD:','<b style="color:black; font-size:1.75vh; line-height: 2.6vh;">')\
                                                .replace('</A><br>\n','</a></b><br>')\
                                                .replace('<a href','<b><a target="_blank" href')\
                                                .replace('</a>','</a></b>')\
                                                .replace('href="ap','href="https://apod.nasa.gov/apod/ap') + '</p>')])
        if clickable:
            card.on_event('click', onclicked)
        return card

    
# APOD search (text can contain AND, OR, etc)
def fullSearch(text):
    response = requests.get('https://apod.nasa.gov/cgi-bin/apod/apod_search?tquery=%s'%text)
    vvv = response.text.replace('<P>','<p>').split('<p>')
    vvv = [x for x in vvv if "Thumbnail image of picture found for this day" in x]
    return vvv


# Given a date string in the "YYYY-MM-DD" format, returns a v.Card for the display of the title + explanation with links
def dateSearch(datestring):
    date = datetime.strptime(datestring,"%Y-%m-%d")
    results = fullSearch(date.strftime("%Y %B %-d"))
    for res in results:
        if date.strftime("%y%m%d") in res:
            return formatResult(res,0,False)
        
        
# Read the full APOD archive and returns a vector containing (datetime,description) for each of the days
def fullArchive():
    # Read HTML of the full archive
    response = requests.get('https://apod.nasa.gov/apod/archivepixFull.html')
    v = response.text.split('\n')

    # Retrieve first and last line containing dates
    slast   = datetime.today().strftime('%Y %B %d')
    sfirst  = 'June 16 1995'
    m = [x for x in v if x.startswith(sfirst)]
    ifirst = v.index(m[0])

    m = [x for x in v if x.startswith(slast)]
    ilast = v.index(m[0])
    
    # Lines containing dates
    vdays = [x for x in v[ilast:ifirst] if ':' in x]

    # Find change of date format
    schange = 'December 31 1999'
    m = [x for x in vdays if x.startswith(schange)]
    ichange = vdays.index(m[0])

    vdays2000 = vdays[:ichange]
    vdays1900 = vdays[ichange:]
    
    days_descr2000 = [(datetime.strptime(x.split(':')[0],'%Y %B %d'), x.split(':')[1].split('>')[1].split('<')[0]) for x in vdays2000]
    days_descr1900 = [(datetime.strptime(x.split(':')[0],'%B %d %Y'), x.split(':')[1].split('>')[1].split('<')[0]) for x in vdays1900]
    days_descr = days_descr2000 + days_descr1900
    return days_descr

full = fullArchive()
#full[100]

In [16]:
# All the output of the panels
outControls    = None
outDescription = None
outPicture     = None
outControls2   = None
outControls3   = None
outPagination  = None
outList        = None


# Click on the title tabs
def on_click_tab(arg):
    global outControls, outDescription, outPicture, outControls2, outControls3, outPagination, outList
    
    g_app.contentResetPanels()
    
    if arg == 'DAY-BY-DAY':
        outControls    = g_app.contentAddPanel('28vw', '85px',   '1vw',  '3vh',  border='1px solid grey', backcolor='#ffffffff', title='Select a date', titlewidth='15.0vh', titlefontsize='1.6vh', titleround=False)
        outDescription = g_app.contentAddPanel('28vw', 'calc(84vh - 140px)', '1vw', 'calc(3vh + 130px)', border='1px solid grey', backcolor='#ffffffff', title='Description', titlewidth='14.0vh', titlefontsize='1.6vh', titleround=False)
        outPicture     = g_app.contentAddPanel('69vw', '85.5vh', '30vw', '3vh',    border='0px solid grey', backcolor='#ffffff00')

        with outControls:
            display(v.Card(class_='pa-0 ma-3', elevation=0, children=[v.Row(children=[space,c1,space,c2,space, dp.draw(), space,c3,space,c4, space,space,c5])]))

        onchangedate()
    elif arg == 'TITLE-SEARCH':
        outControls2 = g_app.contentAddPanel('28vw', '85.5vh', '1vw',  '3vh', border='1px solid grey', backcolor='#ffffffff', title='Search on titles', titlewidth='20.0vh', titlefontsize='1.6vh', titleround=False)
        outPicture   = g_app.contentAddPanel('69vw', '85.5vh', '30vw', '3vh', border='0px solid grey', backcolor='#ffffff00')
        
        with outControls2:
            display(titlefield)
            display(v.Row(justify='center', children=[bsearch.draw(), treeflat.draw()]))
            display(outTree)
    else:
        outControls3  = g_app.contentAddPanel('650px',              '80px',  '1vw', '3vh',               border='1px solid grey', backcolor='#ffffffff', title='Search on full text', titlewidth='20.0vh', titlefontsize='1.6vh', titleround=False)
        outPagination = g_app.contentAddPanel('calc(98vw - 660px',  '80px',  'calc(1vw + 660px)', '3vh', border='1px solid grey', backcolor='#ffffffff', title='Navigate results',    titlewidth='18.0vh', titlefontsize='1.6vh', titleround=False)
        outList       = g_app.contentAddPanel('98vw',   'calc(85vh - 80px)', '1vw', 'calc(3vh + 90px)',  border='0px solid grey', backcolor='#ffffff00')
        
        with outControls3:
            display(v.Row(justify='center',children=[v.Html(tag='div',children=[fullfield], style_='width: 450px; overflow: hidden;'), widgets.VBox([space,space,menusearch]), space, widgets.VBox([space,space,bsearch2.draw()])]))
            
        with outPagination:
            display(widgets.HBox([paglab1.draw(),paglab2.draw(),pagination]))
            
        page_changed()
            
    g_app.show()

    
# Click on the credits text
def on_click_credits():
    g_app.snackbar('CREDITS')
    
# Click on the logo
def on_click_logo():
    g_app.snackbar('LOGO')

# Click on the footer buttons
def on_click_footer(arg):
    g_app.snackbar(arg)

    
# Create the app instance
g_app = app.app(title='Astronomy Picture of the Day from NASA',
                titleheight='6vh',
                titlecredits='Created by Unit I.3 - BDAP',
                titlewidth='44%',
                titlestyle='color: white; font-weight: 500; font-size: 32px;',
                titlespacestyle='width: 1vw;',
                dark=False,
                backgroundimageurl='...',
                backgroundimageposition='bottom left',
                titletabs=['DAY-BY-DAY', 'TITLE-SEARCH', 'FULL-SEARCH'],
                titletabsstile='font-size: 17px; font-weight: bold;',
                titletabscolor=settings.color_second,
                sidepanelwidth=0,
                footertext='NASA.   Each day a different image or photograph of our fascinating universe is featured, along with a brief explanation written by a professional astronomer',
                footercolor=settings.color_first ,
                footerdark=True,
                footerbuttons=[],
                footerheight=42,
                footercopyright=True,
                onclicktab=on_click_tab,
                onclickcredits=on_click_credits,
                onclicklogo=on_click_logo,
                onclickfooter=on_click_footer,
                fullscreen=True)


# Background image
g_app.contentBackground('...')
g_app.contentResetPanels()

    
d = None


# Set the disabled state of the 4 buttons to move back and forward from the current day
def setDisabled(b, days):
    date = datetime.strptime(dp.date, "%Y-%m-%d") + relativedelta(days=days)
    datestr = date.strftime("%Y-%m-%d")
    if datestr >= mindate and datestr <= maxdate: b.disabled = False
    else:                                         b.disabled = True

        
# Display the image in a fullscreen dialog
fullscreen_url = None

def on_click_image(widget, event, data):
    if not fullscreen_url is None:
        ifull = v.Img(src=fullscreen_url, class_='pa-0 ma-0', contain=False, position='center center', width='100vw', height='calc(100vh - 30px)', max_width='100vw', max_height='calc(100vh - 30px)')
        dlg = g_app.dialogGeneric(title='Zoom', show=True, addclosebuttons=True, persistent=False, fullscreen=True, content=[ifull], titleheight='30px')
        

# Selection of a date
def onchangedate():
    global d, fullscreen_url
    
    fullscreen_url = None
    
    outDescription.clear_output()
    outPicture.clear_output()
    
    setDisabled(bminus5, -5)
    setDisabled(bminus1, -1)
    setDisabled(bplus1,   1)
    setDisabled(bplus5,   5)
    btoday.disabled = (dp.date == maxdate)
    
    progr = progress.progress(None, text='Loading...', size=110, width=12, show=False, outputheight=90)
    progr.showAbsolute(g_app.outservice, 'calc(65vw - 110px)', 'calc(50vh - 110px)', 9999)
    

    params = {'api_key': api_key, 'date': dp.date }
    response = requests.get('https://api.nasa.gov/planetary/apod', params=params)
    d = json.loads(response.text)

    
    if 'title' in d:
        title = d['title']
        if 'copyright' in d: copyright = d['copyright']
        else:                copyright = 'Public domain image courtesy of NASA'
        url = d['url']
        description = d['explanation']

        with outDescription:
            cco = v.CardText(children=['Copyright: ' + copyright])
            c = dateSearch(dp.date)
            if c is None:
                cti = v.CardTitle(children=[title])
                cte = v.CardText(children=[description])
                display(v.Card(width='calc(28vw - 26px)', height='calc(83vh - 140px)', elevation=0, children=[cti,cte,cco]))
            else:
                display(v.Card(width='calc(28vw - 26px)', height='calc(83vh - 140px)', elevation=0, children=[c,cco]))

        progr.close()

        if d['media_type'] == 'video':
            with outPicture:
                display(IFrame(url,"99%",800))
        else:
            i = v.Img(src=url, class_='pa-0 ma-0', contain=True, position='center center', width='69vw', height='85.5vh', max_width='69vw', max_height='85.5vh')
            fullscreen_url = url
            i.on_event('click', on_click_image)
            with outPicture:
                display(i)
    else:
        progr.close()

# Creation of controls for day-by-day page
maxdate = datetime.today().strftime('%Y-%m-%d')
mindate = "1995-06-16"
        
dp = datePicker.datePicker(date=None, label='Date', offset_x=True, offset_y=False, onchange=onchangedate,
                           mindate=mindate, maxdate=maxdate)

def onclick(days):
    if days == 0:
        dp.date = maxdate
    else:
        date = datetime.strptime(dp.date, "%Y-%m-%d") + relativedelta(days=days)
        dp.date = date.strftime("%Y-%m-%d")
    
space = v.Html(tag='div',children=[' '], style_='width: 8px; height: 8px; overflow: hidden;')
bminus5 = button.button('', icon='mdi-step-backward-2', iconsmall=True, width=34, height=34, tooltip='Move 5 days backward', iconcolor='white', selected=True, outlined=False, onclick=onclick, argument=-5)
bminus1 = button.button('', icon='mdi-step-backward',   iconsmall=True, width=34, height=34, tooltip='Move 1 days backward', iconcolor='white', selected=True, outlined=False, onclick=onclick, argument=-1)
bplus1  = button.button('', icon='mdi-step-forward',    iconsmall=True, width=34, height=34, tooltip='Move 1 days forward',  iconcolor='white', selected=True, outlined=False, onclick=onclick, argument= 1)
bplus5  = button.button('', icon='mdi-step-forward-2',  iconsmall=True, width=34, height=34, tooltip='Move 5 days forward',  iconcolor='white', selected=True, outlined=False, onclick=onclick, argument= 5)
btoday  = button.button('', icon='mdi-calendar-today',  iconsmall=True, width=34, height=34, tooltip='Move to last image',   iconcolor='white', selected=True, outlined=False, onclick=onclick, argument= 0)

c1 = widgets.VBox([space,space,bminus5.draw()])
c2 = widgets.VBox([space,space,bminus1.draw()])
c3 = widgets.VBox([space,space,bplus1.draw()])
c4 = widgets.VBox([space,space,bplus5.draw()])
c5 = widgets.VBox([space,space,btoday.draw()])

# Creation of controls for title-search page
titlefield = v.TextField(v_model='', label='Insert text to search on pictures\' titles', persistent_hint=True, class_="pa-0 ma-4 mt-8", autofocus=False,
                         clearable=True, height=28, width='28vw', 
                         color=settings.color_first, validate_on_blur=True)

# Selection of a node of the tree
def ontreeselected(arg):
    if treeflat.value:
        v = arg.split(' ')
    else:
        v = arg.split('#')
    year = int(v[0])
    month = list(calendar.month_name).index(v[1])
    day = int(v[2].split(':')[0])
    dp.date = datetime(year,month,day).strftime('%Y-%m-%d')  # calls onchangedate()!

    
# Click on the "search" button
def onsearch(arg=None):
    
    progr = progress.progress(None, text='Loading...', size=110, width=12, show=False, outputheight=90)
    progr.showAbsolute(g_app.outservice, 'calc(65vw - 110px)', 'calc(50vh - 110px)', 9999)
    
    
    outTree.clear_output()
    
    if treeflat.value:
        items = [str(x[0].year) + ' ' + calendar.month_name[x[0].month] + ' ' + str(x[0].day) + ': ' + x[1] for x in full]
    else:
        items = [str(x[0].year) + '#' + calendar.month_name[x[0].month] + '#' + str(x[0].day) + ': ' + x[1] for x in full]

    openall = False

    if not titlefield.v_model is None and len(titlefield.v_model) > 0:
        to_search = titlefield.v_model.lower()
        if len(to_search) > 0:
            items = [x for x in items if to_search in x.lower()]
            if len(items) < 1000:
                openall = True

            treecard = treeview.createTreeviewFromList(items,
                                                       separator='#',
                                                       rootName='Archive',
                                                       displayfullname=False,
                                                       selectable=False,
                                                       activatable=True,
                                                       open_on_click=True,
                                                       iconsshow=True,
                                                       iconsfolder=True,
                                                       width='26vw',
                                                       height='calc(84.5vh - 155px)',
                                                       tooltips=True,
                                                       tooltips_chars=10,
                                                       opened_all=openall,
                                                       on_activated=ontreeselected)
            with outTree:
                display(treecard)
                
    progr.close()
    
    
bsearch = button.button('Search', dark=True, width=210, height=42, tooltip='Launch search of pictures by title',
                        icon='mdi-magnify', iconcolor='white', disabled=False, selected=True, outlined=False, onclick=onsearch)

treeflat = switch.switch(True, "Flat", inset=True, onchange=onsearch)
#treeflat.disabled = True

outTree = widgets.Output(layout=Layout(width='27.7vw', height='calc(85.5vh - 145px)'))


# Creation of controls for full-search page
fullfield = v.TextField(v_model='', label='Insert text for full search', persistent_hint=True, class_="pa-0 ma-2 mt-6 mb-n5", autofocus=False,
                        clearable=True, height=28, width='450px',
                        color=settings.color_first, validate_on_blur=True)


# List of common searches organized in a tree
predefinedSearches = [
    'Stars#Binary Stars', 'Stars#Black Holes', 'Stars#Globular Clusters', 'Stars#Individual Stars', 'Stars#Neutron Stars', 'Stars#Nurseries', 'Stars#Open Clusters', 'Stars#Sun', 'Stars#White Dwarfs',
    'Galaxies#Clusters of Galaxies', 'Galaxies#Colliding Galaxies', 'Galaxies#Elliptical Galaxies', 'Galaxies#Local Group', 'Galaxies#Milky Way', 'Galaxies#Spiral Galaxies',
    'Nebulae#Dark Nebulae', 'Nebulae#Emission Nebulae', 'Nebulae#Planetary Nebulae', 'Nebulae#Reflection Nebulae', 'Nebulae#Supernova Remnants', 'Nebulae#Carina Nebulae', 
    'Miscellaneous#Quasars', 'Miscellaneous#Active Galactic Nuclei', 'Miscellaneous#Dark Matter',
    'Solar System#Sun', 'Solar System#Mercury', 'Solar System#Venus', 'Solar System#Earth', 'Solar System#Earth Moon', 'Solar System#Mars', 'Solar System#Jupiter', 'Solar System#Jupiter Moon', 'Solar System#Saturn', 'Solar System#Saturn Moon', 'Solar System#Uranus', 'Solar System#Neptune', 'Solar System#Pluto',
    'Comets#Hyakutake', 'Comets#Hale-Bopp', 'Comets#Halley',
    'Asteroids',
    'Space Technology#Rockets', 'Space Technology#Launch Vehicles', 'Space Technology#Orbiting Observatories', 'Space Technology#Space Stations', 'Space Technology#Earth Observatories',
    'People#Scientists', 'People#Astronauts',
    'Sky#Messier Objects', 'Sky#Sky Views'
]

def oncommonsearch(arg=None):
    vvv = arg.split('#')
    fullfield.v_model = vvv[-1]
    onsearch2()
    
treesearch = treeview.createTreeviewFromList(predefinedSearches,
                                             separator='#',
                                             selectable=False,
                                             activatable=True,
                                             displayfullname=False,
                                             open_on_click=False,
                                             opened_all=True,
                                             rootName='Common searches',
                                             expand_selection_to_parents=False,
                                             on_activated=oncommonsearch,
                                             width='350px',
                                             height='1500px')
searchHtml = v.Html(tag='div',height='1500px', children=[treesearch]) #, style_='overflow: hidden;')
searchCard = v.Card(width='350px', height='600px', elevation=0, children=[searchHtml]) #, style_='overflow-x: visible;')

    
# Create a popup menu with the tree as children
btnsearch = v.Btn(v_on='menuData.on', color=settings.color_first, fab=True, dark=True, depressed=True,
                  disabled=False, width=40, height=40, rounded=False, outlined=True,
                  children=[v.Icon(small=False, children=['mdi-play'])])

menusearch = v.Menu(offset_y=True, open_on_hover=True, dense=True,
                    v_slots=[{'name': 'activator','variable': 'menuData','children': btnsearch}],
                    children=[searchCard])



results = []
npages  = 0
results_per_page = 4

# Click on the "search" button
def onsearch2(arg=None):
    global results, npages
    
    outList.clear_output()
    
    paglab1.text = '0'
    pagination.total_visible = 12
    
    if not fullfield.v_model is None and len(fullfield.v_model) > 0:
        results = fullSearch(fullfield.v_model)
        
        paglab1.text = str(len(results))
        outPagination.clear_output(wait=True)
        with outPagination:
            display(widgets.HBox([paglab1.draw(),paglab2.draw(),pagination]))
            
        npages = int(float(len(results)+results_per_page-1)/float(results_per_page))
        pagination.length = npages
        if npages > 0:
            pagination.v_model=1
            page_changed()
    
bsearch2 = button.button('Search', dark=True, width=130, height=42, tooltip='Launch search of pictures by full-text',
                         icon='mdi-magnify', iconcolor='white', disabled=False, selected=True, outlined=False, onclick=onsearch2)

# Changed page on the Pagination widget
def page_changed(widget=None, event=None, data=None):
    
    current_page = pagination.v_model - 1
    outList.clear_output()
    if len(results) >= current_page*results_per_page:
        with outList:
            start = current_page*results_per_page
            end   = start + results_per_page
            if end >= len(results): end = len(results)
            for i in range(start,end):
                display(formatResult(results[i]))

pagination = v.Pagination(length=npages, total_visible=12, v_model=1, circle=True, dark=False, elevation=0, color=settings.color_first, class_='pa-0 ma-0 ml-6 mt-4')
pagination.on_event('input', page_changed)

paglab1 = label.label('      0',        textweight=500, margins=2, margintop=7, height=22)
paglab2 = label.label('pictures found', textweight=300, margins=2, margintop=7, height=22)

# Display the first page
on_click_tab('DAY-BY-DAY')
onchangedate()

Dialog(children=[Card(children=[Output(layout=Layout(height='100vh', width='100vw'))])], fullscreen=True, no_c…